# Transparent Fallback

This part of tutorial is also available in step-by-step notebook version on [github](https://github.com/dmlc/minpy/blob/master/examples/tutorials/transparent_fallback.ipynb). Please try it out!


## Concept of transparent fallback

Since MinPy fully integrates MXNet, it allows you to use GPU to speed up your algorithm with only minor change, while keeping the neat NumPy syntax you just went through. However, NumPy is a giant library with hundreds of operators. Our supported GPU operators are only a subset of them, so it is inevitable that you want to try some functions that are currently missing on GPU side. To solve this problem, MinPy designed a policy system to determine which implementation shoud be applied, consisted of build-in policies in `minpy.dispatch.policy` (also aliased in minpy root):

- `PreferMXNetPolicy()` [**Default**]: Prefer MXNet. Use NumPy as a transparent fallback, which wil be discussed below.
- `OnlyNumPyPolicy()`: Only use NumPy operations.
- `OnlyMXNetPolicy()`: Only use MXNet operations.
- `BlacklistPolicy()`: Discussed below.

The default policy `PreferMXNetPolicy` gracefully adopts the NumPy implementation once the operator is missing on GPU side, and handles the memory copies among GPU and CPU for you, whose procedure can be expressed naively as below picture:

![PreferMXNetPolicy](images/PreferMXNetPolicy.png)

The code below will prove this for you.

In [1]:
import minpy.numpy as np
# First turn on the logging to know what happens under the hood.
import logging
logging.getLogger('minpy.array').setLevel(logging.DEBUG)

# x is created as a MXNet array
x = np.zeros((10, 20))


# `cosh` is currently missing in MXNet's GPU implementation.
# So `x` will fallback to a NumPy array, so you will see a 
# logging like "Copy from MXNet array to NumPy array...", then
# NumPy's implementation of `cosh` will be called to get the
# result `y` as a NumPy array. But you don't need to worry 
# about the memory copy from GPU -> CPU
y = np.cosh(x)


# `log` has MXNet's GPU implementation, so it will copy the 
# array `y` from NumPy array to MXNet array and you will see
# a logging like "Copy from NumPy array to MXNet array..."
# Once again, you don't need to worry about it. It is transparent.
z = np.log(y)


# Turn off the logging.
logging.getLogger('minpy.array').setLevel(logging.WARN)

I1107 15:47:11 4755 minpy.array:_synchronize_data:415] Copy from MXNet array to NumPy array for Array "4407168528" of shape (10L, 20L).
I1107 15:47:11 4755 minpy.array:_synchronize_data:421] Copy from NumPy array to MXNet array for Array "4407300256" of shape (10, 20).


However, there exist a few of NumPy functions cannot work property in the `PreferMXNetPolicy`, due to the difference between NumPy and MXNet interface. As the code below:

In [2]:
# In PreferMXNetPolicy, np.random.normal will redirect to MXNet's implementation
# but it does not support mu and sigma to be arrays (only scalar
# is supported right now).
def gaussian_cluster_generator(num_samples=10000, num_features=500, num_classes=5):
    mu = np.random.rand(num_classes, num_features)
    sigma = np.ones((num_classes, num_features)) * 0.1
    num_cls_samples = num_samples / num_classes
    x = np.zeros((num_samples, num_features))
    y = np.zeros((num_samples, num_classes))
    for i in range(num_classes):
        # this line will occur an error
        cls_samples = np.random.normal(mu[i,:], sigma[i,:], (num_cls_samples, num_features))
        x[i*num_cls_samples:(i+1)*num_cls_samples] = cls_samples
        y[i*num_cls_samples:(i+1)*num_cls_samples,i] = 1
    return x, y

gaussian_cluster_generator(10000, 500, 5)

MXNetError: Invalid Parameter format for loc expect float but value='<mxnet.ndarray.NDArray object at 0x106b2f450>'

So we design another blacklist machinism for you. The operator in the blacklist will fallback to its numpy implementaiton and the content of blacklist will be prepared when you install MinPy automatically. This will solve most of these problem. If you encounter another one, please feel free to raise an issue in our github! The procedure of function call under `PerferMXNetPolicy` will become:

![Blacklist](images/PreferMXNetPolicyWithBlacklist.png)

Although we've provided some function to you for switching policy easily (see [Select Policy for Operations](https://minpy.readthedocs.io/en/latest/feature/policy.html)), you really don't need to consider this unless you meet a rare situation that current policy doesn't work properly, which will be discussed in ["Pitfalls when working together with NumPy"](http://minpy.readthedocs.io/en/latest/feature/limitation.html). 